In [ ]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Maximum Find

### FloodingUpdate
* *Raspodijeljeni* algoritmi se implementiraju kao podklase od `NodeAlgorithm`. Raspodijeljeni algoritmi mogu koristiti samo informacije spremljene u samoj memoriji čvora te one dobivene očitanjem osjetila koje pojeduje čvor.

Svaki node prosljeđuje poruku svojim susjedima dokle god informacija koju primi osvježuju njegovo znanje odnosno zadovoljava uvijet prosljeđivanja. 

In [ ]:
from pymote.message import Message
from pymote.algorithm import NodeAlgorithm

class FloodingUpdate(NodeAlgorithm):
    """
    This is modified Flooding algorithm (Santoro2007 p.13) so that every node
    continues to forward flood messages as long as information gathered is updating its knowledge.
    Note: does not have global termination detection
    """
    required_params = ('dataKey',)
    default_params = {'neighborsKey':'Neighbors','msgKey':'MessageCount'}
    
    def initializer(self):
        """ Starts in every node satisfying initiator condition. """
        for node in self.network.nodes():
            node.memory[self.neighborsKey] = node.compositeSensor.read()['Neighbors'] #removing sender from destination
            if self.initiator_condition(node):
                self.network.outbox.insert(0, Message(destination=node,
                                                     header=NodeAlgorithm.INI))
                node.memory['msgKey']=0
                node.status = 'FLOODING'
            
            

    def flooding(self, node, message):
        if message.header == NodeAlgorithm.INI:
            node.send(Message(header='Flood',
                              data=self.initiator_data(node)))
            node.memory['msgKey'] = node.memory['msgKey']+ len(list(node.memory[self.neighborsKey]))

        if message.header == 'Flood':
            updated_data = self.handle_flood_message(node, message)
            if updated_data:
                destination_nodes = list(node.memory[self.neighborsKey])
                destination_nodes.remove(message.source) # send to every neighbor-sender
                
                node.memory['msgKey']=node.memory['msgKey']+len(destination_nodes)
                node.send(Message(destination=destination_nodes,header='Flood',
                                  data=updated_data))

    def initiator_condition(self, node):
        raise NotImplementedError

    def initiator_data(self, node):
        raise NotImplementedError

    def handle_flood_message(self, node, message):
        raise NotImplementedError

    STATUS = {'FLOODING': flooding,  # init,term
              }


### Temperature Sensor

In [ ]:
from pymote.sensor import Sensor
import random
class TempSensor(Sensor):

    def read(self,node):
        return {'Temperature': random.randrange(-15,40,1)}


### MaxFind
Ukoliko primi poruku s temperaturom većom od njemu trenutno poznate najveće temperature prosljeđuje poruku susjedima.

In [ ]:
class MaxFind(FloodingUpdate):

    default_params = {'temperatureKey':'Temperature','neighborsKey': 'Neighbors','maxKey':'Max'}
    def initiator_condition(self, node):
        #svi će biti iniciatori        
        node.compositeSensor=(TempSensor,'Temperature')
        node.memory[self.temperatureKey]=node.compositeSensor.read()['Temperature']
        node.memory[self.maxKey]=node.memory[self.temperatureKey]      
        #trenutna najveća je njegova, jedina koju ima    
        return node.memory[self.maxKey] is not None

    def initiator_data(self, node):
        #node.memory[self.msgKey] = 1
        return node.memory[self.maxKey]
        #ovdje ne smije biti read, jer bi svaki puta očitao drugačiju temperaturu
                      
    def handle_flood_message(self, node, message):
        
        if message.data > node.memory[self.maxKey]:
            node.memory[self.maxKey]=message.data
            #node.memory[self.msgKey]=node.memory[self.msgKey]+1
            #node.memory[self.msgKey]=node.memory[self.msgKey]+
            return message.data


    STATUS = {'FLOODING':FloodingUpdate.STATUS.get('FLOODING'),  # init,term 
             }

## Eksperimentalna analiza

In [ ]:
from pymote.networkgenerator import NetworkGenerator
from pymote.network import Network
from pymote.npickle import write_pickle, read_pickle

### Random Mreža

In [ ]:
net_gen = NetworkGenerator(15)
net = net_gen.generate_random_network()
write_pickle(net,'mrezaRandom.tar.gz')

In [ ]:
net.show() #po gašenju izaziva Dead Kernel

### Mreža Lanac (n = 4, m = 5)

In [ ]:
net = Network()
n = 5
u = 100
for i in range(0,n*u,u):
    node = net.add_node(pos=[1,i])
    node.commRange=110
    
write_pickle(net,'mrezaList.tar.gz')

### Mreža Zvijezda (n = 4, m = 5)

In [ ]:
net = Network()
n = 3
for i in range(100,(n+1)*100,100):
    node= net.add_node(pos=[i,i])
    node.commRange=200

node= net.add_node(pos=[100,300])
node.commRange=200
node= net.add_node(pos=[300,100])
node.commRange=200

write_pickle(net,'mrezaZvijezda.tar.gz')

## Simulacija

In [1]:
#Import Classes- classes defind in notebook won't be used
from pymote.algorithms.turk2018.UpdateMax import MaxFind

In [2]:
from pymote.networkgenerator import NetworkGenerator
from pymote.network import Network
from pymote.npickle import write_pickle, read_pickle

from pymote.simulation import Simulation

#net = read_pickle('mrezaRandom.tar.gz') 
net = read_pickle('mrezaList.tar.gz')
#net = read_pickle('mrezaZvijezda.tar.gz')

net.algorithms = ( (MaxFind, {'dataKey':'D'}), )
sim = Simulation(net)
sim.run()

suma=0

for node in net.nodes():
    print node.memory['Max']
    print node.memory
    print node.status
    suma=suma+node.memory['msgKey']
print "Number of nodes:"
print len(list(net.nodes()))
print "Number of edges:"
print len(list(net.edges()))
print "Total number of messages:"
print suma

DEBUG   [simulation.py]: Simulation 0x10a4fe18 created successfully.


DEBUG:pymote.simulation:Simulation 0x10a4fe18 created successfully.


WARNING [__init__.py]: Environment variable PYMOTE_SETTINGS_MODULE is undefined, using global_settings.


DEBUG   [simulation.py]: [MaxFind] Step 1 finished


DEBUG:pymote.simulation:[MaxFind] Step 1 finished


...

DEBUG   [simulation.py]: [MaxFind] Algorithm finished


DEBUG:pymote.simulation:[MaxFind] Algorithm finished


INFO    [simulation.py]: Simulation has finished. There are no algorithms left to run. To run it from the start use sim.reset().


INFO:pymote.simulation:Simulation has finished. There are no algorithms left to run. To run it from the start use sim.reset().


33
{'Neighbors': [<Node id=187>], 'Max': 33, 'Temperature': 11, 'msgKey': 1}
FLOODING
33
{'Neighbors': [<Node id=188>, <Node id=186>], 'Max': 33, 'Temperature': 30, 'msgKey': 3}
FLOODING
33
{'Neighbors': [<Node id=187>, <Node id=189>], 'Max': 33, 'Temperature': 10, 'msgKey': 4}
FLOODING
33
{'Neighbors': [<Node id=188>, <Node id=190>], 'Max': 33, 'Temperature': -12, 'msgKey': 3}
FLOODING
33
{'Neighbors': [<Node id=189>], 'Max': 33, 'Temperature': 33, 'msgKey': 1}
FLOODING
Number of nodes:
5
Number of edges:
4
Total number of messages:
12
